<a href="https://colab.research.google.com/github/farihast/be5210_finalproject/blob/viraj/Viraj_Sampat_BE5210_Final_Project_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages and Datasets

In [ ]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig
from pathlib import Path
from sklearn.model_selection import train_test_split
import scipy
#from scipy.fft import fft, rfft, irfft, fftfreq, rfftfreq
from scipy.interpolate import CubicSpline
from scipy.io import savemat
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/BE5210_FinalProject/leaderboard_data.mat', 'rb') as g:
  leaderboard_data = scipy.io.loadmat('/content/drive/MyDrive/BE5210_FinalProject/leaderboard_data.mat')
leaderboard_ecog = leaderboard_data['leaderboard_ecog']

In [ ]:
with open('/content/drive/MyDrive/BE5210_FinalProject/raw_training_data.mat', 'rb') as g:
  raw_training_data = scipy.io.loadmat('/content/drive/MyDrive/BE5210_FinalProject/raw_training_data.mat')
train_dg = raw_training_data['train_dg']
train_ecog = raw_training_data['train_ecog']

In [ ]:
print(leaderboard_data.keys())
print(raw_training_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'leaderboard_ecog'])
dict_keys(['__header__', '__version__', '__globals__', 'train_dg', 'train_ecog'])


In [ ]:
print(leaderboard_data['__globals__'], leaderboard_data['__version__'], leaderboard_data['__header__'])
print(raw_training_data['__globals__'], raw_training_data['__version__'], raw_training_data['__header__'])

[] 1.0 b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Thu Apr  9 21:34:40 2020'
[] 1.0 b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Apr 10 12:52:24 2020'


In [ ]:
print(leaderboard_ecog.shape, train_dg.shape, train_ecog.shape)

(3, 1) (3, 1) (3, 1)


In [ ]:
train_dg[0][0].shape

(300000, 5)

In [ ]:
train_ecog[1][0].shape

(300000, 48)

In [ ]:
train_ecog[0][0] = np.delete(train_ecog[0][0], 54, 1)
#train_ecog[0][0] = train_ecog[0][0][:,np.array([0, 39, 42, 20, 1, 41, 22, 43, 12])]
train_ecog[1][0] = np.delete(train_ecog[1][0], 37, 1)
train_ecog[1][0] = np.delete(train_ecog[1][0], 20, 1)

leaderboard_ecog[0][0] = np.delete(leaderboard_ecog[0][0], 54, 1)
#leaderboard_ecog[0][0] = leaderboard_ecog[0][0][:,np.array([0, 39, 42, 20, 1, 41, 22, 43, 12])]
leaderboard_ecog[1][0] = np.delete(leaderboard_ecog[1][0], 37, 1)
leaderboard_ecog[1][0] = np.delete(leaderboard_ecog[1][0], 20, 1)

In [ ]:
print(train_ecog[0][0].shape)
print(leaderboard_ecog[0][0].shape)

(300000, 61)
(147500, 61)


In [ ]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(train_ecog[0][0], train_dg[0][0], test_size=0.10, shuffle = False)
X_train2, X_test2, y_train2, y_test2 = train_test_split(train_ecog[1][0], train_dg[1][0], test_size=0.10, shuffle = False)
X_train3, X_test3, y_train3, y_test3 = train_test_split(train_ecog[2][0], train_dg[2][0], test_size=0.10, shuffle = False)

print(X_train1.shape[0])
print(X_test1.shape[0])

270000
30000


In [ ]:
leader_x1 = leaderboard_ecog[0][0]
leader_x2 = leaderboard_ecog[1][0]
leader_x3 = leaderboard_ecog[2][0]

#Filter Data

In [ ]:
def filter_data(raw_eeg, fs=1000):
  """
  Write a filter function to clean underlying data.
  Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
  Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
  distorting the underlying data!

  Input: 
    raw_eeg (samples x channels): the raw signal
    fs: the sampling rate (1000 for this dataset)
  Output: 
    clean_data (samples x channels): the filtered signal
  """
  filter = sig.firwin(50, [1, 400], fs = fs, pass_zero = 'bandpass')
  #filter = sig.firwin(251, [60, 200], fs = fs, pass_zero = False)
  filtered_data = sig.filtfilt(filter, [1], raw_eeg, axis = 0)
  return filtered_data

In [ ]:
filt_sig = filter_data(X_train1)
print(filt_sig.shape)

(270000, 61)


#Windows

In [ ]:
'''
def num_windows(x, fs,winLen, winDisp):
  xLen = len(x)
  winLen_vals = winLen * fs
  winDisp_vals =winDisp * fs
  return int((xLen - winLen_vals) / winDisp_vals ) +1

def MovingWinFeats(x, fs, winLen, winDisp, featFn):
  xLen = len(x)
  #NumWins(fs, winLen, winDisp, x)
  numWins = num_windows(fs, winLen, winDisp, x)
  result = []
  wd_fs = winDisp*fs
  wl_fs = winLen*fs
  for i in range(numWins):
    finish = (xLen - 1) - wd_fs *i 
    start = finish - wl_fs
    win = x[int(start):int(finish)]
    result.append(featFn(win))
  result.reverse()
  return result

'''
def num_windows(x, fs, winLen, winDisp):
  xLen = len(x)  # length of signal [samples]
  return (int)((xLen - winLen*fs) / (winDisp*fs) + 1)
'''
def num_windows(x, fs,winLen, winDisp):
  xLen = len(x)
  winLen_vals = winLen * fs
  winDisp_vals =winDisp * fs
  return int((xLen - winLen_vals) / winDisp_vals ) +1
'''
def MovingWinFeats(x, fs, winLen, winDisp, featFn):
  xLen = len(x)
  #NumWins(fs, winLen, winDisp, x)
  numWins = num_windows(x, fs,winLen, winDisp)
  result = []
  wd_fs = winDisp*fs
  wl_fs = winLen*fs
  for i in range(numWins):
    finish = (xLen - 1) - wd_fs *i 
    start = finish - wl_fs
    win = x[int(start):int(finish)]
    result.append(featFn(win))
  result.reverse()
  return result


In [ ]:
wins = num_windows(y_train1, 1000, .200, .025)
print(wins)


10793


In [ ]:
def AvgFreqDomainMagnitude(x, bandrange):
  yf = np.abs(np.fft.fft(x, axis = 0))
  xf = np.fft.fftfreq(len(x[:,0]), 1/1000)

  indices = np.where((xf >= bandrange[0]) & (xf <= bandrange[1]))[0]
  #return np.sum(np.square(yf[indices]))
  #if np.isnan(np.mean(yf[indices,:], axis=0)).any():
  #  return 0
  #else:  
  return np.nanmean(yf[indices,:], axis=0)

In [ ]:
def AvgVoltageTimeDomain(x):
  return np.mean(x, axis = 0)

In [ ]:
'''
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window. 
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!
    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """
  number_of_features = 6
  channels = filtered_window.shape[1]
  features = np.zeros((channels, number_of_features))

  filter_1 = sig.firwin(50, [5, 15], fs = fs, pass_zero = 'bandpass')
  filter_2 = sig.firwin(50, [20, 25], fs = fs, pass_zero = 'bandpass')
  filter_3 = sig.firwin(50, [75, 115], fs = fs, pass_zero = 'bandpass')
  filter_4 = sig.firwin(50, [125, 160], fs = fs, pass_zero = 'bandpass')
  filter_5 = sig.firwin(50, [160, 175], fs = fs, pass_zero = 'bandpass')
  filt_sig_1 = sig.filtfilt(filter_1, [1], filtered_window, padtype = None, axis = 1)
  filt_sig_2 = sig.filtfilt(filter_2, [1], filtered_window, padtype = None, axis = 1)
  filt_sig_3 = sig.filtfilt(filter_3, [1], filtered_window, padtype = None, axis = 1)
  filt_sig_4 = sig.filtfilt(filter_4, [1], filtered_window, padtype = None, axis = 1)
  filt_sig_5 = sig.filtfilt(filter_5, [1], filtered_window, padtype = None, axis = 1)

  for i in range(channels):
    signal = filtered_window[:,i]
    ll = np.sum(np.absolute(np.ediff1d(signal)))
    area = np.sum(np.abs(signal))
    energy = np.sum(np.square(signal))
    
    mean = np.mean(signal)
    val = []
    for j in range(len(signal)-1):
      above = (signal[j] - mean > 0) and (signal[j+1] - mean < 0)
      below = (signal[j] - mean < 0) and (signal[j+1] - mean > 0)
      val.append(above or below)
    z_cross = np.sum(val)

    filt_feat_1 = np.sum(np.square(filt_sig_1[i,:]))

    filt_feat_2 = np.sum(np.square(filt_sig_2[i,:]))

    filt_feat_3 = np.sum(np.square(filt_sig_3[i,:]))

    filt_feat_4 = np.sum(np.square(filt_sig_4[i,:]))

    filt_feat_5 = np.sum(np.square(filt_sig_5[i,:]))


    #features[i,0] = ll
    #features[i,1] = area
    #features[i,2] = energy
    #features[i,3] = z_cross
    features[i,0] = mean
    features[i,1] = filt_feat_1
    features[i,2] = filt_feat_2
    features[i,3] = filt_feat_3
    features[i,4] = filt_feat_4
    features[i,5] = filt_feat_5
  return features

'''
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window. 
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!

    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """
  wind_samples, num_channels = filtered_window.shape
  numFeatures = 6
  features = np.zeros((num_channels, numFeatures))

  #for i in range(num_channels):
  features[:, 0] = AvgVoltageTimeDomain(filtered_window)
  features[:, 1] = AvgFreqDomainMagnitude(filtered_window, [5, 15])
  features[:, 2] = AvgFreqDomainMagnitude(filtered_window, [5, 75])
  features[:, 3] = AvgFreqDomainMagnitude(filtered_window, [75, 115])
  features[:, 4] = AvgFreqDomainMagnitude(filtered_window, [125, 160])
  features[:, 5] = AvgFreqDomainMagnitude(filtered_window, [160, 175])

  return features

In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
  """
    Write a function which processes data through the steps of filtering and
    feature calculation and returns features. Points will be awarded for completing
    each step appropriately (note that if one of the functions you call within this script
    returns a bad output, you won't be double penalized). Note that you will need
    to run the filter_data and get_features functions within this function. 

    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """
  numWins = num_windows(raw_ecog, fs, window_length, window_overlap)
  num_samples, num_channels = raw_ecog.shape
  numFeatures = 6
  all_feats = np.zeros((numWins, num_channels * numFeatures))

  filtered = filter_data(raw_ecog)
  #filtered = raw_ecog
  for i in range(numWins):
    window = filtered[(int)(i * window_overlap * fs) : (int)((i*window_overlap + window_length)*fs)]
    all_feats[i] = get_features(window, fs=fs).flatten()

  return all_feats


In [ ]:
R_test_path = Path('/content/drive/MyDrive/testRfunction.pkl')
with open(R_test_path, 'rb') as f:
  R_test = pickle.load(f)

In [ ]:
'''
def create_R_matrix(features, N_wind):
  """ 
  Write a function to calculate the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  m = features.shape[0]
  n = features.shape[1]
  R = np.zeros((m, n*N_wind))
  first_rows = features[0:N_wind-1, :]
  features = np.insert(features, 0, first_rows, axis = 0)
  preceding = N_wind-1
  for i in range(0,m):
    features_window = np.transpose(features[i:i+N_wind,:]).flatten()
    R[i,:] = features_window
  R = np.insert(R, 0, 1, axis = 1)
  return R
'''
def create_R_matrix(features, N_wind):
  """ 
  Write a function to calculate the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  num_samples, chanxfeat = features.shape
  R = np.zeros((num_samples, N_wind*chanxfeat))
  
  # append first N-1 (2) rows to beginning of features
  features = np.vstack((features[0:N_wind-1, :], features))

  for i in range(num_samples):
    R[i] = features[i:i+N_wind, :].flatten()

  R = np.hstack((np.ones((num_samples,1)), R))
  
  return R

In [ ]:
print(np.mean(create_R_matrix(R_test, 3)))

25.46678519830894


In [ ]:
window_length = 400
window_displacement = 25

start_index = int(1/2*window_length)

down_sampled_glove_train_1 = y_train1[start_index::window_displacement]

down_sampled_glove_test_1 = y_test1[start_index::window_displacement]

down_sampled_glove_train_2 = y_train2[start_index::window_displacement]
down_sampled_glove_test_2 = y_test2[start_index::window_displacement]

down_sampled_glove_train_3 = y_train3[start_index::window_displacement]
down_sampled_glove_test_3 = y_test3[start_index::window_displacement]

'''
for i in range(len(down_sampled_glove_train_1)-1):
  if (len(y_train1) - (start_index + i*window_displacement) <= window_length):
    down_sampled_glove_train_1 = down_sampled_glove_train_1[0:i]
    down_sampled_glove_test_1 = down_sampled_glove_test_1[0:i]
    down_sampled_glove_train_2 = down_sampled_glove_train_2[0:i]
    down_sampled_glove_test_2 = down_sampled_glove_test_2[0:i]
    down_sampled_glove_train_3 = down_sampled_glove_train_3[0:i]
    down_sampled_glove_test_3 = down_sampled_glove_test_3[0:i]
    break
'''
cutoff = len(down_sampled_glove_train_1) - num_windows(y_train1, 1000, window_length/1000, window_displacement/1000)
if (cutoff != 0):
  down_sampled_glove_train_1 = down_sampled_glove_train_1[0:-cutoff]
  down_sampled_glove_test_1 = down_sampled_glove_test_1[0:-cutoff]
  down_sampled_glove_train_2 = down_sampled_glove_train_2[0:-cutoff]
  down_sampled_glove_test_2 = down_sampled_glove_test_2[0:-cutoff]
  down_sampled_glove_train_3 = down_sampled_glove_train_3[0:-cutoff]
  down_sampled_glove_test_3 = down_sampled_glove_test_3[0:-cutoff]



feats_1 = get_windowed_feats(X_train1, 1000, window_length/1000, window_displacement/1000)
feats_2 = get_windowed_feats(X_train2, 1000, window_length/1000, window_displacement/1000)
feats_3 = get_windowed_feats(X_train3, 1000, window_length/1000, window_displacement/1000)

n = 8
R_1 = create_R_matrix(feats_1, n)
R_2 = create_R_matrix(feats_2, n)
R_3 = create_R_matrix(feats_3, n)


f_1 = np.linalg.pinv(R_1.T@R_1)@(R_1.T@down_sampled_glove_train_1)
f_2 = np.linalg.pinv(R_2.T@R_2)@(R_2.T@down_sampled_glove_train_2)
f_3 = np.linalg.pinv(R_3.T@R_3)@(R_3.T@down_sampled_glove_train_3)

feats_test_1 = get_windowed_feats(X_test1, 1000, window_length/1000, window_displacement/1000)
R_1_test = create_R_matrix(feats_test_1, n)

feats_test_2 = get_windowed_feats(X_test2, 1000, window_length/1000, window_displacement/1000)
R_2_test = create_R_matrix(feats_test_2, n)

feats_test_3 = get_windowed_feats(X_test3, 1000, window_length/1000, window_displacement/1000)
R_3_test = create_R_matrix(feats_test_3, n)

In [ ]:
pred_1 = R_1_test @ f_1

pred_2 = R_2_test @ f_2

pred_3 = R_3_test @ f_3

correlation_scores_interpolated = np.zeros((5,3))

ending_index = len(y_test1) - np.arange(len(y_test1))[start_index::window_displacement][0:-cutoff][-1]

lin_pred_1 = np.zeros((len(y_test1),5))
lin_pred_2 = np.zeros((len(y_test1),5))
lin_pred_3 = np.zeros((len(y_test1),5))

for i in range(5):
  subject_1_cs = CubicSpline(np.arange(len(y_test1))[start_index::window_displacement][0:-cutoff], pred_1[:,i])
  subject_1_interpolated = subject_1_cs(np.arange(len(y_test1))[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  lin_pred_1[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(len(y_test2))[start_index::window_displacement][0:-cutoff], pred_2[:,i])
  subject_2_interpolated = subject_2_cs(np.arange(len(y_test2))[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  lin_pred_2[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(len(y_test3))[start_index::window_displacement][0:-cutoff], pred_3[:,i])
  subject_3_interpolated = subject_3_cs(np.arange(len(y_test3))[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  lin_pred_3[:,i] = subject_3_interpolated

  correlation_scores_interpolated[i,0] = pearsonr(y_test1[:,i], subject_1_interpolated).statistic
  correlation_scores_interpolated[i,1] = pearsonr(y_test2[:,i], subject_2_interpolated).statistic
  correlation_scores_interpolated[i,2] = pearsonr(y_test3[:,i], subject_3_interpolated).statistic


correlation_scores = np.zeros((5,3))

for i in range(5):
  correlation_scores[i,0] = pearsonr(down_sampled_glove_test_1[:,i], pred_1[:,i]).statistic
  correlation_scores[i,1] = pearsonr(down_sampled_glove_test_2[:,i], pred_2[:,i]).statistic
  correlation_scores[i,2] = pearsonr(down_sampled_glove_test_3[:,i], pred_3[:,i]).statistic

#correlation_scores = np.delete(correlation_scores, 3, 0)

#correlation_scores_interpolated = np.delete(correlation_scores_interpolated, 3, 0)



#print('Correlation Scores for the test finger angle predictions using the optimal \
#linear decoder model. The rows represent fingers 1-3 and 5, and the columns represent \
#subjects 1-3.')
#print(correlation_scores)
#print('The mean correlation score across fingers (except 4th finger) and subjects for the optimal linear decoder model:')
#print(np.mean(correlation_scores))


print('Correlation Scores for the interpolated test finger angle predictions using the optimal \
linear decoder model. The rows represent fingers 1-3 and 5, and the columns represent \
subjects 1-3.')
print(np.delete(correlation_scores_interpolated, 3, 0))
print('The mean correlation score across all fingers and subjects for the optimal linear decoder model:')
print(np.mean(np.delete(correlation_scores_interpolated, 3, 0)))

#print(np.mean(correlation_scores_interpolated, axis=0))


Correlation Scores for the interpolated test finger angle predictions using the optimal linear decoder model. The rows represent fingers 1-3 and 5, and the columns represent subjects 1-3.
[[ 0.46381992  0.32992841  0.47115731]
 [ 0.50831934  0.27994763  0.41726258]
 [ 0.09341172  0.1293929   0.55356225]
 [-0.07696221  0.18840562  0.340644  ]]
The mean correlation score across all fingers and subjects for the optimal linear decoder model:
0.30824078914592784


In [ ]:
leader_feats_test_1 = get_windowed_feats(leader_x1, 1000, window_length/1000, window_displacement/1000)
leader_R_1_test = create_R_matrix(leader_feats_test_1, n)

leader_feats_test_2 = get_windowed_feats(leader_x2, 1000, window_length/1000, window_displacement/1000)
leader_R_2_test = create_R_matrix(leader_feats_test_2, n)

leader_feats_test_3 = get_windowed_feats(leader_x3, 1000, window_length/1000, window_displacement/1000)
leader_R_3_test = create_R_matrix(leader_feats_test_3, n)

leader_lin_pred_1 = leader_R_1_test @ f_1

leader_lin_pred_2 = leader_R_2_test @ f_2

leader_lin_pred_3 = leader_R_3_test @ f_3


In [ ]:
leader_lin_subject_1_interpolated = np.zeros((147500,5))
leader_lin_subject_2_interpolated = np.zeros((147500,5))
leader_lin_subject_3_interpolated = np.zeros((147500,5))

ending_index = leader_x1.shape[0] - np.arange(leader_x1.shape[0])[start_index::window_displacement][0:-cutoff][-1]

for i in range(5):
  subject_1_cs = CubicSpline(np.arange(leader_x1.shape[0])[start_index::window_displacement][0:-cutoff], leader_lin_pred_1[:,i])
  subject_1_interpolated = subject_1_cs(np.arange(leader_x1.shape[0])[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  leader_lin_subject_1_interpolated[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(leader_x2.shape[0])[start_index::window_displacement][0:-cutoff], leader_lin_pred_2[:,i])
  subject_2_interpolated = subject_2_cs(np.arange(leader_x2.shape[0])[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  leader_lin_subject_2_interpolated[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(leader_x3.shape[0])[start_index::window_displacement][0:-cutoff], leader_lin_pred_3[:,i])
  subject_3_interpolated = subject_3_cs(np.arange(leader_x3.shape[0])[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  leader_lin_subject_3_interpolated[:,i] = subject_3_interpolated

In [ ]:
'''
#SVR Model
from sklearn.svm import SVR

svr_1 = SVR(kernel = 'rbf', epsilon= 0.5, C = 3)
svr_2 = SVR(kernel = 'rbf', epsilon = 0.5, C = 1)
svr_3 = SVR(kernel = 'rbf', epsilon = 0.7, C = 1)

svr_pred_1 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))
svr_pred_2 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))
svr_pred_3 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))

leader_svr_pred_1 = np.zeros((leader_feats_test_1.shape[0],5))
leader_svr_pred_2 = np.zeros((leader_feats_test_2.shape[0],5))
leader_svr_pred_3 = np.zeros((leader_feats_test_3.shape[0],5))

for i in range(5):
  svr_1.fit(feats_1,down_sampled_glove_train_1[:,i])
  svr_train_score = svr_1.score(feats_1,down_sampled_glove_train_1[:,i])
  svr_predict = svr_1.predict(feats_test_1)
  svr_pred_1[:,i] = svr_predict

  leader_svr_predict = svr_1.predict(leader_feats_test_1)
  leader_svr_pred_1[:,i] = leader_svr_predict

for i in range(5):
  svr_2.fit(feats_2,down_sampled_glove_train_2[:,i])
  svr_train_score = svr_2.score(feats_2,down_sampled_glove_train_2[:,i])
  svr_predict = svr_2.predict(feats_test_2)
  svr_pred_2[:,i] = svr_predict

  leader_svr_predict = svr_2.predict(leader_feats_test_2)
  leader_svr_pred_2[:,i] = leader_svr_predict

for i in range(5):
  svr_3.fit(feats_3,down_sampled_glove_train_3[:,i])
  svr_train_score = svr_3.score(feats_3,down_sampled_glove_train_3[:,i])
  svr_predict = svr_3.predict(feats_test_3)
  svr_pred_3[:,i] = svr_predict

  leader_svr_predict = svr_3.predict(leader_feats_test_3)
  leader_svr_pred_3[:,i] = leader_svr_predict


svr_correlation_scores_interpolated = np.zeros((5,3))
svr_subject_1_interpolated = np.zeros((len(y_test1),5))
svr_subject_2_interpolated = np.zeros((len(y_test2),5))
svr_subject_3_interpolated = np.zeros((len(y_test3),5))

# Test SVR Interpolation
for i in range(5):
  subject_1_cs = CubicSpline(np.arange(len(y_test1))[start_index::window_displacement][0:-cutoff], svr_pred_1[:,i])
  subject_1_interpolated = subject_1_cs(np.arange(len(y_test1))[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  svr_subject_1_interpolated[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(len(y_test2))[start_index::window_displacement][0:-cutoff], svr_pred_2[:,i])
  subject_2_interpolated = subject_2_cs(np.arange(len(y_test2))[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  svr_subject_2_interpolated[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(len(y_test3))[start_index::window_displacement][0:-cutoff], svr_pred_3[:,i])
  subject_3_interpolated = subject_3_cs(np.arange(len(y_test3))[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  svr_subject_3_interpolated[:,i] = subject_3_interpolated

  svr_correlation_scores_interpolated[i,0] = pearsonr(y_test1[:,i], subject_1_interpolated).statistic
  svr_correlation_scores_interpolated[i,1] = pearsonr(y_test2[:,i], subject_2_interpolated).statistic
  svr_correlation_scores_interpolated[i,2] = pearsonr(y_test3[:,i], subject_3_interpolated).statistic

# Leader SVR Interpolation
leader_svr_pred_1_interpolated = np.zeros((147500,5))
leader_svr_pred_2_interpolated = np.zeros((147500,5))
leader_svr_pred_3_interpolated = np.zeros((147500,5))

for i in range(5):
  subject_1_cs = CubicSpline(np.arange(leader_x1.shape[0])[start_index::window_displacement][0:-cutoff], leader_svr_pred_1[:,i])
  subject_1_interpolated = subject_1_cs(np.arange(leader_x1.shape[0])[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  leader_svr_pred_1_interpolated[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(leader_x2.shape[0])[start_index::window_displacement][0:-cutoff], leader_svr_pred_2[:,i])
  subject_2_interpolated = subject_2_cs(np.arange(leader_x2.shape[0])[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  leader_svr_pred_2_interpolated[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(leader_x3.shape[0])[start_index::window_displacement][0:-cutoff], leader_svr_pred_3[:,i])
  subject_3_interpolated = subject_3_cs(np.arange(leader_x3.shape[0])[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  leader_svr_pred_3_interpolated[:,i] = subject_3_interpolated
'''

"\n#SVR Model\nfrom sklearn.svm import SVR\n\nsvr_1 = SVR(kernel = 'rbf', epsilon= 0.5, C = 3)\nsvr_2 = SVR(kernel = 'rbf', epsilon = 0.5, C = 1)\nsvr_3 = SVR(kernel = 'rbf', epsilon = 0.7, C = 1)\n\nsvr_pred_1 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))\nsvr_pred_2 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))\nsvr_pred_3 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))\n\nleader_svr_pred_1 = np.zeros((leader_feats_test_1.shape[0],5))\nleader_svr_pred_2 = np.zeros((leader_feats_test_2.shape[0],5))\nleader_svr_pred_3 = np.zeros((leader_feats_test_3.shape[0],5))\n\nfor i in range(5):\n  svr_1.fit(feats_1,down_sampled_glove_train_1[:,i])\n  svr_train_score = svr_1.score(feats_1,down_sampled_glove_train_1[:,i])\n  svr_predict = svr_1.predict(feats_test_1)\n  svr_pred_1[:,i] = svr_predict\n\n  leader_svr_predict = svr_1.predict(leader_feats_test_1)\n  leader_svr_pred_1[:,i] = leader_svr_predict\n\nfor i in range(5):\n  svr_2.fit(feats_2,down_sampled_glove_train_2[:,i])\n  

In [ ]:
'''
#svr_correlation_scores_interpolated = np.delete(svr_correlation_scores_interpolated, 3, 0)
print('Correlation Scores for the test finger angle predictions using the SVR \
model. The rows represent fingers 1-3, 5 and the columns represent \
subjects 1-3.')
print(np.delete(svr_correlation_scores_interpolated, 3, 0))
#print(np.mean(svr_correlation_scores_interpolated, axis=0))
print('The mean correlation score across fingers and subjects for the SVR model:')
print(np.mean(np.delete(svr_correlation_scores_interpolated, 3, 0)))
'''

"\n#svr_correlation_scores_interpolated = np.delete(svr_correlation_scores_interpolated, 3, 0)\nprint('Correlation Scores for the test finger angle predictions using the SVR model. The rows represent fingers 1-3, 5 and the columns represent subjects 1-3.')\nprint(np.delete(svr_correlation_scores_interpolated, 3, 0))\n#print(np.mean(svr_correlation_scores_interpolated, axis=0))\nprint('The mean correlation score across fingers and subjects for the SVR model:')\nprint(np.mean(np.delete(svr_correlation_scores_interpolated, 3, 0)))\n"

In [ ]:
#Lasso Model

# lasso_1 = LassoCV(cv = 8,random_state = 3)
# lasso_2 = LassoCV(cv = 8,random_state = 3)
# lasso_3 = LassoCV(cv = 8,random_state = 3)

lasso_1 = Lasso(alpha = 10)
lasso_2 = Lasso(alpha = 10)
lasso_3 = Lasso(alpha = 10)

lasso_pred_1 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))
lasso_pred_2 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))
lasso_pred_3 = np.zeros((len(down_sampled_glove_test_1[:,1]),5))

leader_lasso_pred_1 = np.zeros((leader_feats_test_1.shape[0],5))
leader_lasso_pred_2 = np.zeros((leader_feats_test_2.shape[0],5))
leader_lasso_pred_3 = np.zeros((leader_feats_test_3.shape[0],5))

lasso_1_coeff = []
lasso_2_coeff = []
lasso_3_coeff = []

for i in range(5):
  lasso_1.fit(feats_1,down_sampled_glove_train_1[:,i])
  lasso_train_score = lasso_1.score(feats_1,down_sampled_glove_train_1[:,i])
  lasso_predict = lasso_1.predict(feats_test_1)
  lasso_pred_1[:,i] = lasso_predict
  lasso_coeff = lasso_1.coef_
  lasso_1_coeff.append(lasso_coeff)

  leader_lasso_predict = lasso_1.predict(leader_feats_test_1)
  leader_lasso_pred_1[:,i] = leader_lasso_predict

for i in range(5):
  lasso_2.fit(feats_2,down_sampled_glove_train_2[:,i])
  lasso_train_score = lasso_2.score(feats_2,down_sampled_glove_train_2[:,i])
  lasso_predict = lasso_2.predict(feats_test_2)
  lasso_pred_2[:,i] = lasso_predict

  lasso_coeff = lasso_2.coef_
  lasso_2_coeff.append(lasso_coeff)

  leader_lasso_predict = lasso_2.predict(leader_feats_test_2)
  leader_lasso_pred_2[:,i] = leader_lasso_predict

for i in range(5):
  lasso_3.fit(feats_3,down_sampled_glove_train_3[:,i])
  lasso_train_score = lasso_3.score(feats_3,down_sampled_glove_train_3[:,i])
  lasso_predict = lasso_3.predict(feats_test_3)
  lasso_pred_3[:,i] = lasso_predict

  lasso_coeff = lasso_3.coef_
  lasso_3_coeff.append(lasso_coeff)

  leader_lasso_predict = lasso_3.predict(leader_feats_test_3)
  leader_lasso_pred_3[:,i] = leader_lasso_predict


lasso_correlation_scores_interpolated = np.zeros((5,3))
lasso_subject_1_interpolated = np.zeros((len(y_test1),5))
lasso_subject_2_interpolated = np.zeros((len(y_test2),5))
lasso_subject_3_interpolated = np.zeros((len(y_test3),5))

# Test Lasso Interpolation
for i in range(5):
  subject_1_cs = CubicSpline(np.arange(len(y_test1))[start_index::window_displacement][0:-cutoff], lasso_pred_1[:,i])
  subject_1_interpolated = subject_1_cs(np.arange(len(y_test1))[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  lasso_subject_1_interpolated[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(len(y_test2))[start_index::window_displacement][0:-cutoff], lasso_pred_2[:,i])
  subject_2_interpolated = subject_2_cs(np.arange(len(y_test2))[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  lasso_subject_2_interpolated[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(len(y_test3))[start_index::window_displacement][0:-cutoff], lasso_pred_3[:,i])
  subject_3_interpolated = subject_3_cs(np.arange(len(y_test3))[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  lasso_subject_3_interpolated[:,i] = subject_3_interpolated

  lasso_correlation_scores_interpolated[i,0] = pearsonr(y_test1[:,i], subject_1_interpolated).statistic
  lasso_correlation_scores_interpolated[i,1] = pearsonr(y_test2[:,i], subject_2_interpolated).statistic
  lasso_correlation_scores_interpolated[i,2] = pearsonr(y_test3[:,i], subject_3_interpolated).statistic

# Leader Lasso Interpolation
leader_lasso_pred_1_interpolated = np.zeros((147500,5))
leader_lasso_pred_2_interpolated = np.zeros((147500,5))
leader_lasso_pred_3_interpolated = np.zeros((147500,5))

for i in range(5):
  subject_1_cs = CubicSpline(np.arange(leader_x1.shape[0])[start_index::window_displacement][0:-cutoff], leader_lasso_pred_1[:,i])
  subject_1_interpolated = subject_1_cs(np.arange(leader_x1.shape[0])[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  leader_lasso_pred_1_interpolated[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(leader_x2.shape[0])[start_index::window_displacement][0:-cutoff], leader_lasso_pred_2[:,i])
  subject_2_interpolated = subject_2_cs(np.arange(leader_x2.shape[0])[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  leader_lasso_pred_2_interpolated[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(leader_x3.shape[0])[start_index::window_displacement][0:-cutoff], leader_lasso_pred_3[:,i])
  subject_3_interpolated = subject_3_cs(np.arange(leader_x3.shape[0])[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  leader_lasso_pred_3_interpolated[:,i] = subject_3_interpolated


print('Correlation Scores for the test finger angle predictions using the Lasso \
model. The rows represent fingers 1-3, 5 and the columns represent \
subjects 1-3.')
print(lasso_correlation_scores_interpolated)
#print(np.mean(svr_correlation_scores_interpolated, axis=0))
print('The mean correlation score across fingers and subjects for the Lasso model:')
print(np.mean(np.delete(lasso_correlation_scores_interpolated, 3, 0)))


Correlation Scores for the test finger angle predictions using the Lasso model. The rows represent fingers 1-3, 5 and the columns represent subjects 1-3.
[[ 0.56304822  0.34476562  0.46168878]
 [ 0.61110238  0.26304442  0.41769809]
 [ 0.06104667  0.09592911  0.47732934]
 [ 0.53857579  0.48972747  0.55956249]
 [-0.15162241  0.0710071   0.38244287]]
The mean correlation score across fingers and subjects for the Lasso model:
0.2997900155411907


In [ ]:
'''
print(correlation_scores_interpolated)
print(svr_correlation_scores_interpolated)
final_test_correlation_scores = np.zeros((5,3))
final_test_prediction_1 = np.zeros((len(y_test1),5))
final_test_prediction_2 = np.zeros((len(y_test2),5))
final_test_prediction_3 = np.zeros((len(y_test3),5))
final_leader_prediction_1 = np.zeros((147500,5))
final_leader_prediction_2 = np.zeros((147500,5))
final_leader_prediction_3 = np.zeros((147500,5))

for i in range(5):
  for j in range(3):
    linear_score = correlation_scores_interpolated[i][j]
    svr_score = svr_correlation_scores_interpolated[i][j]
    if j==0:
      if True:#linear_score >= svr_score:
        final_test_prediction_1[:,i] = lin_pred_1[:,i]
        final_leader_prediction_1[:,i] = leader_lin_subject_1_interpolated[:,i]
      else:
        final_test_prediction_1[:,i] = svr_subject_1_interpolated[:,i]
        final_leader_prediction_1[:,i] = leader_svr_pred_1_interpolated[:,i]
    if j==1:
      if True:#linear_score >= svr_score:
        final_test_prediction_2[:,i] = lin_pred_2[:,i]
        final_leader_prediction_2[:,i] = leader_lin_subject_2_interpolated[:,i]
      else:
        final_test_prediction_2[:,i] = svr_subject_2_interpolated[:,i]
        final_leader_prediction_2[:,i] = leader_svr_pred_2_interpolated[:,i]
    if j==2:
      if True:#linear_score >= svr_score:
        final_test_prediction_3[:,i] = lin_pred_3[:,i]
        final_leader_prediction_3[:,i] = leader_lin_subject_3_interpolated[:,i]
      else:
        final_test_prediction_3[:,i] = svr_subject_3_interpolated[:,i]
        final_leader_prediction_3[:,i] = leader_svr_pred_3_interpolated[:,i]
      
for i in range(5):     
  final_test_correlation_scores[i,0] = pearsonr(y_test1[:,i], final_test_prediction_1[:,i]).statistic
  final_test_correlation_scores[i,1] = pearsonr(y_test2[:,i], final_test_prediction_2[:,i]).statistic
  final_test_correlation_scores[i,2] = pearsonr(y_test3[:,i], final_test_prediction_3[:,i]).statistic

final_test_correlation_scores = np.delete(final_test_correlation_scores, 3, 0)
print('Correlation Scores for the test finger angle predictions using the Final \
model. The rows represent fingers 1-3, 5 and the columns represent \
subjects 1-3.')
print(final_test_correlation_scores)
#print(np.mean(svr_correlation_scores_interpolated, axis=0))
print('The mean correlation score across fingers and subjects for the final model:')

print(np.mean(final_test_correlation_scores))
'''

"\nprint(correlation_scores_interpolated)\nprint(svr_correlation_scores_interpolated)\nfinal_test_correlation_scores = np.zeros((5,3))\nfinal_test_prediction_1 = np.zeros((len(y_test1),5))\nfinal_test_prediction_2 = np.zeros((len(y_test2),5))\nfinal_test_prediction_3 = np.zeros((len(y_test3),5))\nfinal_leader_prediction_1 = np.zeros((147500,5))\nfinal_leader_prediction_2 = np.zeros((147500,5))\nfinal_leader_prediction_3 = np.zeros((147500,5))\n\nfor i in range(5):\n  for j in range(3):\n    linear_score = correlation_scores_interpolated[i][j]\n    svr_score = svr_correlation_scores_interpolated[i][j]\n    if j==0:\n      if True:#linear_score >= svr_score:\n        final_test_prediction_1[:,i] = lin_pred_1[:,i]\n        final_leader_prediction_1[:,i] = leader_lin_subject_1_interpolated[:,i]\n      else:\n        final_test_prediction_1[:,i] = svr_subject_1_interpolated[:,i]\n        final_leader_prediction_1[:,i] = leader_svr_pred_1_interpolated[:,i]\n    if j==1:\n      if True:#linea

In [ ]:

new_correlation_scores_interpolated = np.zeros((5,3))

ending_index = len(y_test1) - np.arange(len(y_test1))[start_index::window_displacement][0:-cutoff][-1]

new_lin_pred_1 = np.zeros((len(y_test1),5))
new_lin_pred_2 = np.zeros((len(y_test1),5))
new_lin_pred_3 = np.zeros((len(y_test1),5))

new_leader_lin_pred_1 = np.zeros((147500,5))
new_leader_lin_pred_2 = np.zeros((147500,5))
new_leader_lin_pred_3 = np.zeros((147500,5))

for i in range(5):
  num_ind = 30

  ind_1 = np.argpartition(np.abs(lasso_1_coeff[i]), -num_ind)[-num_ind:]
  reduced_feats_1 = feats_1[:,np.array(ind_1)]

  ind_2 = np.argpartition(np.abs(lasso_2_coeff[i]), -num_ind)[-num_ind:]
  reduced_feats_2 = feats_2[:,np.array(ind_2)]

  ind_3 = np.argpartition(np.abs(lasso_3_coeff[i]), -num_ind)[-num_ind:]
  reduced_feats_3 = feats_3[:,np.array(ind_3)]

  reduced_R_1 = create_R_matrix(reduced_feats_1, n)
  reduced_R_2 = create_R_matrix(reduced_feats_2, n)
  reduced_R_3 = create_R_matrix(reduced_feats_3, n)

  reduced_f_1 = np.linalg.pinv(reduced_R_1.T@reduced_R_1)@(reduced_R_1.T@down_sampled_glove_train_1[:,i])
  reduced_f_2 = np.linalg.pinv(reduced_R_2.T@reduced_R_2)@(reduced_R_2.T@down_sampled_glove_train_2[:,i])
  reduced_f_3 = np.linalg.pinv(reduced_R_3.T@reduced_R_3)@(reduced_R_3.T@down_sampled_glove_train_3[:,i])

  reduced_feats_test_1 = feats_test_1[:,np.array(ind_1)]
  reduced_R_1_test = create_R_matrix(reduced_feats_test_1, n)

  reduced_feats_test_2 = feats_test_2[:,np.array(ind_2)]
  reduced_R_2_test = create_R_matrix(reduced_feats_test_2, n)

  reduced_feats_test_3 = feats_test_3[:,np.array(ind_3)]
  reduced_R_3_test = create_R_matrix(reduced_feats_test_3, n)

  new_pred_1 = reduced_R_1_test @ reduced_f_1
  new_pred_2 = reduced_R_2_test @ reduced_f_2
  new_pred_3 = reduced_R_3_test @ reduced_f_3


  subject_1_cs = CubicSpline(np.arange(len(y_test1))[start_index::window_displacement][0:-cutoff], new_pred_1)
  subject_1_interpolated = subject_1_cs(np.arange(len(y_test1))[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  new_lin_pred_1[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(len(y_test2))[start_index::window_displacement][0:-cutoff], new_pred_2)
  subject_2_interpolated = subject_2_cs(np.arange(len(y_test2))[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  new_lin_pred_2[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(len(y_test3))[start_index::window_displacement][0:-cutoff], new_pred_3)
  subject_3_interpolated = subject_3_cs(np.arange(len(y_test3))[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  new_lin_pred_3[:,i] = subject_3_interpolated

  new_correlation_scores_interpolated[i,0] = pearsonr(y_test1[:,i], subject_1_interpolated).statistic
  new_correlation_scores_interpolated[i,1] = pearsonr(y_test2[:,i], subject_2_interpolated).statistic
  new_correlation_scores_interpolated[i,2] = pearsonr(y_test3[:,i], subject_3_interpolated).statistic

  #### ADD LEADER NEW LINEAR PREDICTION HERE ######
  reduced_leader_feats_test_1 = leader_feats_test_1[:,np.array(ind_1)]
  reduced_leader_R_1_test = create_R_matrix(reduced_leader_feats_test_1, n)

  reduced_leader_feats_test_2 = leader_feats_test_2[:,np.array(ind_2)]
  reduced_leader_R_2_test = create_R_matrix(reduced_leader_feats_test_2, n)

  reduced_leader_feats_test_3 = leader_feats_test_3[:,np.array(ind_3)]
  reduced_leader_R_3_test = create_R_matrix(reduced_leader_feats_test_3, n)

  reduced_leader_lin_pred_1 = reduced_leader_R_1_test @ reduced_f_1

  reduced_leader_lin_pred_2 = reduced_leader_R_2_test @ reduced_f_2

  reduced_leader_lin_pred_3 = reduced_leader_R_3_test @ reduced_f_3

  subject_1_cs = CubicSpline(np.arange(leader_x1.shape[0])[start_index::window_displacement][0:-cutoff], reduced_leader_lin_pred_1)
  subject_1_interpolated = subject_1_cs(np.arange(leader_x1.shape[0])[start_index:-ending_index])
  subject_1_interpolated = np.insert(subject_1_interpolated, 0, subject_1_interpolated[0]*np.ones(start_index))
  subject_1_interpolated = np.insert(subject_1_interpolated, -1, subject_1_interpolated[-1]*np.ones(ending_index))
  new_leader_lin_pred_1[:,i] = subject_1_interpolated

  subject_2_cs = CubicSpline(np.arange(leader_x2.shape[0])[start_index::window_displacement][0:-cutoff], reduced_leader_lin_pred_2)
  subject_2_interpolated = subject_2_cs(np.arange(leader_x2.shape[0])[start_index:-ending_index])
  subject_2_interpolated = np.insert(subject_2_interpolated, 0, subject_2_interpolated[0]*np.ones(start_index))
  subject_2_interpolated = np.insert(subject_2_interpolated, -1, subject_2_interpolated[-1]*np.ones(ending_index))
  new_leader_lin_pred_2[:,i] = subject_2_interpolated

  subject_3_cs = CubicSpline(np.arange(leader_x3.shape[0])[start_index::window_displacement][0:-cutoff], reduced_leader_lin_pred_3)
  subject_3_interpolated = subject_3_cs(np.arange(leader_x3.shape[0])[start_index:-ending_index])
  subject_3_interpolated = np.insert(subject_3_interpolated, 0, subject_3_interpolated[0]*np.ones(start_index))
  subject_3_interpolated = np.insert(subject_3_interpolated, -1, subject_3_interpolated[-1]*np.ones(ending_index))
  new_leader_lin_pred_3[:,i] = subject_3_interpolated
  #################################################


print('Correlation Scores for the interpolated test finger angle predictions using the NEW optimal \
linear decoder model. The rows represent fingers 1-3 and 5, and the columns represent \
subjects 1-3.')

print(np.delete(new_correlation_scores_interpolated, 3, 0))

print('The mean correlation score across fingers (except 4th finger) and subjects for the optimal linear decoder model:')
print(np.mean(np.delete(new_correlation_scores_interpolated, 3, 0)))


Correlation Scores for the interpolated test finger angle predictions using the NEW optimal linear decoder model. The rows represent fingers 1-3 and 5, and the columns represent subjects 1-3.
[[ 0.59311563  0.52385068  0.61623904]
 [ 0.68314247  0.26891068  0.50654776]
 [ 0.18045884  0.20226109  0.67348992]
 [-0.32979627  0.15394554  0.5092506 ]]
The mean correlation score across fingers (except 4th finger) and subjects for the optimal linear decoder model:
0.38178466429097924


In [ ]:
#create an example submission array
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = new_leader_lin_pred_1
predictions[1,0] = new_leader_lin_pred_2
predictions[2,0] = new_leader_lin_pred_3
#save the array using the right format
savemat('predictions.mat', {'predicted_dg':predictions})